In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from numpy import argmax
# from pickle import load
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import load_model
# # from nltk.translate.bleu_score import corpus_bleu
import json
# import random
import csv


# SEED = 10
# #IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/CNNFeatureVectors/'
# IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/one_sample_cnn/'
# NUM_IMAGE_EMBEDDING_CHUNKS = 1
# GLOVE_EMBEDDING_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/glove.6B.300d.txt'
# MAX_SEQUENCE_LENGTH = 92
# WORD_EMBEDDING_DIM = 300
# CAPTION_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/image_to_caption.csv'
# filepath = '/content/drive/My Drive/Colab_Notebooks/DL_data/model-ep{epoch:03d}-loss{loss:.3f}.h5'


SEED = 10
IMAGE_EMBEDDING_DIR = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/'
NUM_IMAGE_EMBEDDING_CHUNKS = 1
GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'
MAX_SEQUENCE_LENGTH = 92
WORD_EMBEDDING_DIM = 300
CAPTION_FILE_NAME = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/image_to_caption.csv'

#PreProcess Captions

In [3]:
def getCaptions(id_list):
    caption_dict = {}
    with open(CAPTION_FILE_NAME) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if row[1] in id_list:
                caption_dict[row[1]] = ['startseq ' + row[2] + ' endseq']
                #caption_dict[row[1]] = 'startseq ' + row[2] + ' endseq'
    return caption_dict

In [4]:
def vocab_fun(captions):
  index_to_word = {}
  word_to_index = {}
  all_words = {}
  for img_id, cap in captions.items():
      for c in cap:
          for word in c.split():
            all_words[word] = 1
  all_vocab=[w for w in all_words]
  index = 0
  for word in all_vocab:
      word_to_index[word] = index
      index_to_word[index] = word
      index += 1
  return (all_vocab, word_to_index, index_to_word)

#Preprocess images

In [5]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 
    
def getImageEmbedding():
    image_embedding = {}
    for i in range (NUM_IMAGE_EMBEDDING_CHUNKS):
        file_name = IMAGE_EMBEDDING_DIR + 'group_'+str(i+1)+'.json'
        with open(file_name) as json_file:
            print(file_name)
            json_data = json.load(json_file)
            json_data = json.loads(json_data)
            image_embedding = Merge(image_embedding, json_data) 
            image_embedding = json_data 
        return image_embedding

#for fit_generator:

In [6]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch): #descriptions are captions
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split() if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photos[key])
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

#Use Prev to get captions and images and pre_process them

In [7]:
image_embd =  getImageEmbedding()
print(len(image_embd))
# for val in image_embd.values():
#   print(np.shape(val))
#   break
image_ids = list(image_embd.keys())
caption_map = getCaptions(image_ids)
all_vocab, wordtoix, ixtoword=vocab_fun(caption_map)
#tokenizer = create_tokenizer(caption_map)

vocab_size = len(all_vocab)#len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = 94 #max_length(caption_map)
#X1train, X2train, ytrain = create_sequences(tokenizer, max_length, caption_map, image_embd, vocab_size)
#X1test, X2test, ytest = create_sequences(tokenizer, max_length, caption_map, image_embd, vocab_size)

/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/group_1.json
4480
Vocabulary Size: 4435


#Embedding Matrix

In [8]:
#get matrxi embedding for glove
embeddings_index = {} # empty dictionary
f = open(GLOVE_EMBEDDING_FILE_NAME, encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
embedding_dim = 300

# Get 300-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


#Building Model

In [9]:
def build_model():
  inputs1 = Input(shape=(2048,),name='images')
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256, activation='relu')(fe1)
  inputs2 = Input(shape=(max_length,),name='sequences')
  se1 = Embedding(vocab_size, WORD_EMBEDDING_DIM, mask_zero=True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)
  decoder1 = add([fe2, se3])
  decoder2 = Dense(256, activation='relu')(decoder1)
  outputs = Dense(vocab_size, activation='softmax')(decoder2)
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  return model

#Training

In [10]:
model=build_model()
model.summary()
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequences (InputLayer)          (None, 94)           0                                            
__________________________________________________________________________________________________
images (InputLayer)             (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 94, 300)      1330500     sequences[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           images[0][0]                    

In [ ]:
batch_size=556
epochs = 2
steps = len(caption_map)
for i in range(epochs):
    generator = data_generator(caption_map, image_embd, wordtoix, max_length, batch_size)
    model.fit_generator(generator, epochs=2, steps_per_epoch=steps, verbose=1)
    model.save('/content/drive/My Drive/Colab_Notebooks/DL_data/late_late_night_model_' + str(i) + '.h5')

Epoch 1/2
  28/4480 [..............................] - ETA: 33:48:22 - loss: 5.3477

In [0]:
import os.path as osp
import os
from pprint import pprint
from skimage.transform import rescale, resize
from skimage import data, color, io
import skimage
import PIL
import scipy
import json
import os.path
from os import path
import tensorflow as tf

from keras.applications.xception import preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import decode_predictions
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
def load_image(image_path,target_size):
    img = skimage.io.imread(image_path)
    image_resized = skimage.transform.resize(img, target_size, anti_aliasing=True)
    return image_resized

def load_cnn_model():
    model = Xception()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    return model

def extract_features_from_images(image_path):
    model = load_cnn_model()
    if path.exists(image_path):
        print(image_path)
        image = load_image(image_path, target_size=(299, 299))
        if image.shape == (299, 299, 3):
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            print(feature)
            return feature

def generate_desc(model, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[word] for word in in_text.split() if word in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        # map integer to word
        word = ixtoword[yhat]
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [85]:
image_path='/content/drive/My Drive/Colab_Notebooks/DL_data/images/1.jpg'
feature=extract_features_from_images(image_path)
print(generate_desc(model, feature, max_length))



91889664/91884032 [==============================] - 7s 0us/step
/content/drive/My Drive/Colab_Notebooks/DL_data/images/1.jpg
[[0.         0.         0.19784988 ... 0.1310169  0.         0.        ]]
startseq the family is having a nice time playing games endseq


In [86]:
image_path='/content/drive/My Drive/Colab_Notebooks/DL_data/images/2.jpg'
feature=extract_features_from_images(image_path)
print(generate_desc(model, feature, max_length))

/content/drive/My Drive/Colab_Notebooks/DL_data/images/2.jpg
[[0.         0.         0.15794557 ... 0.13588832 0.         0.        ]]
startseq the family is having a nice time playing games endseq
